In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/makart/work/hse/nlp/cinderella_telegram


In [19]:
import itertools

import pandas as pd
import torch
from nltk import TweetTokenizer
from sklearn.neighbors import NearestNeighbors

from src.datamodules.tg_messages_datamodule import TgMessagesDataModule, read_dfs
from src.models.tg_messages_module import TgCinderellaLitModule
from src.models.components.text_linear_model import TextLinearModel
from src.utils.utils import prepare_vocab

In [4]:
data = pd.read_csv("data/default/train_data.csv", index_col="id")
solution = pd.read_csv("data/default/train_solution.csv", index_col="id")
train = data.join(solution, on="id")

In [5]:
datamodule = TgMessagesDataModule()

In [6]:
datamodule.setup()

self.vocab_size=18619


In [7]:
item = datamodule.data_train[0]
item

{'id': 272250,
 'message': [1, 1341, 9, 633, 5, 165, 40, 181, 196, 2, 37, 852, 181],
 'category': 2}

In [8]:
train_dataloader = datamodule.train_dataloader()

In [9]:
batch = next(iter(train_dataloader))
batch

(tensor([2, 1, 0, 2, 2, 0, 1, 2, 1, 0, 1, 0, 0, 1, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2,
         0, 0, 1, 1, 0, 1, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 2, 0, 2, 0, 1, 0, 1,
         2, 0, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0]),
 tensor([  7, 397,  63,  ...,  43,  23,   3]),
 tensor([   0,  171,  334,  345,  503,  520,  527,  574,  626,  632,  639,  689,
          695,  700,  722,  755,  760,  768,  773,  811,  820, 1107, 1108, 1111,
         1234, 1236, 1254, 1377, 1478, 1479, 1481, 1527, 1663, 1928, 2009, 2100,
         2142, 2179, 2181, 2682, 2766, 2811, 2825, 2831, 2888, 2906, 3351, 3364,
         3647, 3658, 3664, 3972, 3975, 4213, 4531, 4756, 4770, 4900, 5126, 5128,
         5148, 5157, 5165, 5179]),
 tensor([275275, 272479, 273883, 274865, 273698, 275666, 273950, 273641, 275054,
         272338, 275103, 272307, 271910, 274161, 274197, 273466, 274056, 273449,
         275564, 274618, 272275, 274786, 274308, 274840, 273864, 272886, 274792,
         274098, 272910, 274682, 273115, 274885, 

In [10]:
net = TextLinearModel(20000, 1024, 3, 4)

In [11]:
model = TgCinderellaLitModule(net=net, optimizer=None, scheduler=None)

/home/makart/work/hse/nlp/nlp_venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: DeprecationWarning: From v0.10 an `'Binary*'`, `'Multiclass*', `'Multilabel*'` version now exist of each classification metric. Moving forward we recommend using these versions. This base metric will still work as it did prior to v0.10 until v0.11. From v0.11 the `task` argument introduced in this metric will be required and the general order of arguments may change, such that this metric will just function as an single entrypoint to calling the three specialized versions.
  warnings.warn(*args, **kwargs)


In [12]:
sd = torch.load('./logs/train/runs/2022-10-20_18-40-17/checkpoints/epoch_005.ckpt')

In [13]:
model.load_state_dict(sd['state_dict'])

<All keys matched successfully>

In [35]:
results = []
for labels, texts, offsets, ids in train_dataloader:
    results.extend(model.net.embedding(texts, offsets))
results = torch.stack(results, axis=0).detach().numpy()

In [15]:
offsets

tensor([  0,  73,  79, 277])

In [16]:
tokenizer = TweetTokenizer()
df_train, df_test = read_dfs("data/")

def tokenize_preprocess(text: str) -> str:
    return " ".join(tokenizer.tokenize(text.lower()))

vocab = prepare_vocab(
    itertools.chain(*(tokenize_preprocess(message).split() for message in df_train.message))
)

In [39]:
emb = model.net.embedding(torch.tensor(vocab(["my", "future"])), torch.tensor([0]))
emb = emb.detach().numpy()

In [40]:
nn = NearestNeighbors().fit(results)

In [42]:
nn.kneighbors(emb, 1, return_distance=False)[0, 0]

3202

In [47]:
" ".join(vocab.lookup_tokens(datamodule.data_train[3202]["message"]))

"the current average funding rate of bitcoin hit a new high in six months , about 0.0532 % . in addition , today's panic and greed index has reached 84 , which is also a record high in six months , and is in a stage of extreme greed ."